In [1]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'

In [2]:
%pwd

'/home/nas4_user/hojuncho/kyudan/AnomLLM/AnomLLM/notebook'

In [3]:
%cd ..
%cd src

/home/nas4_user/hojuncho/kyudan/AnomLLM/AnomLLM
/home/nas4_user/hojuncho/kyudan/AnomLLM/AnomLLM/src


/home/nas4_user/hojuncho/.cache/pypoetry/virtualenvs/anomllm-yItPHLz9-py3.10/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [4]:
%pwd

'/home/nas4_user/hojuncho/kyudan/AnomLLM/AnomLLM/src'

In [5]:
#Please edit the BASE_DIR on your situation.

BASE_DIR = "/home/nas4_user/hojuncho/kyudan/AnomLLM/AnomLLM"

In [6]:
from prompt import time_series_to_image
from utils import view_base64_image, display_messages, collect_results
from data.synthetic import SyntheticDataset

data_name = 'point'
data_dir = f'{BASE_DIR}/data/synthetic/{data_name}/eval/'
train_dir = f'{BASE_DIR}/data/synthetic/{data_name}/train/'
eval_dataset = SyntheticDataset(data_dir)
eval_dataset.load()
train_dataset = SyntheticDataset(train_dir)
train_dataset.load()

Loaded dataset point with 400 series.
Loaded dataset point with 400 series.


In [7]:
from utils import plot_series_and_predictions
from utils import view_base64_image, display_messages, collect_results
idx = 0
directory = f'{BASE_DIR}/results/synthetic/{data_name}'
series = eval_dataset[idx][1].numpy() # eval dataset의 문제 series.
anomaly_locations = eval_dataset[idx][0].numpy() # eval dataset의 gt가 될 것.


In [8]:
raw_results = collect_results(directory, raw=True)
series = {k: v[idx] for k, v in raw_results.items()}


In [9]:
results = collect_results(directory)
results = {k: v[idx] for k, v in results.items()}

In [10]:
from utils import interval_to_vector
gt = interval_to_vector(anomaly_locations[0])
# interval_to_vector 를 하면 gt의 구간에 대해서 1 로 바뀜.
# 0 0 0 0 0 0 1 1 1 0 0 0 0 0 

In [11]:
import pandas as pd
from sklearn.metrics import precision_score, recall_score
from affiliation.generics import convert_vector_to_events
from affiliation.metrics import pr_from_events
from utils import compute_metrics

data = {}

for name, prediction in results.items():
    metrics = compute_metrics(gt, prediction)
    data[name] = metrics

df = pd.DataFrame(data).T
df


,precision,recall,f1,affi precision,affi recall,affi f1
gemini-1.5-flash (localization (rawData)),0.0,0.0,0.0,0.182,0.197,0.189
gemini-1.5-flash (0shot-text),0.0,0.0,0.0,0.316,0.317,0.316


In [12]:
# Keep only the first four columns
df = df.iloc[:, :4].fillna(0)

In [13]:
from utils import highlight_by_ranking
from IPython.core.display import HTML
styled_df = highlight_by_ranking(df.T).T
HTML(styled_df.to_html(escape=False))


,precision,recall,f1,affi precision
gemini-1.5-flash (localization (rawData)),0.00,0.00,0.00,18.20
gemini-1.5-flash (0shot-text),0.00,0.00,0.00,31.60


In [14]:
results = collect_results(directory)

In [ ]:
import numpy as np
import pandas as pd
from tqdm import trange
from utils import compute_metrics, interval_to_vector

# Initialize dictionary to store results
metric_names = ['precision', 'recall', 'f1', 'affi precision', 'affi recall', 'affi f1']

results_dict = {
    key: [[] for _ in metric_names] for key in results.keys()
}

# Initialize DataFrame to store results
df_list = []

for i in trange(0, 400):
    series = eval_dataset[idx][1].numpy()
    anomaly_locations = eval_dataset[idx][0].numpy()
    gt = interval_to_vector(anomaly_locations[0])

    for name, prediction in results.items():
        metrics = compute_metrics(gt, prediction[i])
        for idx, metric_name in enumerate(metric_names):
            results_dict[name][idx].append(metrics[metric_name])

# Convert dictionary to DataFrame
df = pd.DataFrame({k: np.mean(v, axis=1) for k, v in results_dict.items()}, 
                  index=['precision', 'recall', 'f1', 'affi precision', 'affi recall', 'affi f1'])

df.T

In [15]:
styled_df = highlight_by_ranking(df).T
# To visualize the output in Jupyter Notebook
from IPython.core.display import HTML
HTML(styled_df.to_html(escape=False))

/home/nas4_user/hojuncho/kyudan/AnomLLM/AnomLLM/src/utils.py:235: RuntimeWarning: invalid value encountered in double_scalars
  ratio = (value - min_val) / (midpoint - min_val)


,gemini-1.5-flash (localization (rawData)),gemini-1.5-flash (0shot-text)
PRE,0.00,0.00
REC,0.00,0.00
F1,0.00,0.00
affi PRE,18.20,31.60


In [16]:
from utils import styled_df_to_latex

latex_table = styled_df_to_latex(styled_df, "Point Anomalies in Sine Wave", label="point_exp")
print(latex_table)

\begin{longtable}{lrr}
\caption{Point Anomalies in Sine Wave} \label{tab:point_exp} \\
\toprule
& \small\fontfamily{cmtt}\selectfont{gemini-1.5-flash} & \small\fontfamily{cmtt}\selectfont{gemini-1.5-flash} \\
& \small\fontfamily{cmtt}\selectfont{(localization (rawData))} & \small\fontfamily{cmtt}\selectfont{(0shot-text)} \\ \endfirsthead
\multicolumn{3}{c}{\tablename\ \thetable\ -- continued from previous page} \\
\toprule
& \small\fontfamily{cmtt}\selectfont{gemini-1.5-flash} & \small\fontfamily{cmtt}\selectfont{gemini-1.5-flash} \\
& \small\fontfamily{cmtt}\selectfont{(localization (rawData))} & \small\fontfamily{cmtt}\selectfont{(0shot-text)} \\ \endhead
\midrule \multicolumn{3}{r}{Continued on next page} \\ \endfoot
\bottomrule \endlastfoot
\midrule
\footnotesize\fontfamily{cmtt}\selectfont{PRE} & \cellcolor{gray!15}\cellcolor{gray!15}\color[RGB]{0,255,0}\small{0.00} & \cellcolor{gray!15}\color[RGB]{0,255,0}\small{0.00} \\
\footnotesize\fontfamily{cmtt}\selectfont{REC} & \color[RGB